<center>

<h1 style="text-align:center"> Higher Order Programming </h1>
<h2 style="text-align:center"> CS3100 Fall 2019 </h2>
</center>

## Review

### Last time

* Pattern Matching

### Today

* New Idioms and library functions.
  + Map, Reduce and Other higher order functions.

## Double and Square

In [ ]:
let double x = 2 * x
let square x = x * x

In [ ]:
double 10

In [ ]:
square 2

## Quad and Fourth

In [ ]:
let quad x = 2 * 2 * x
let fourth x = (x * x) * (x * x)

In [ ]:
quad 10

In [ ]:
fourth 2

## Quad and Fourth

Abstract away the details using `double` and `square`.

In [ ]:
let quad x = double (double x)

In [ ]:
quad 10

In [ ]:
let fourth x = square (square x)

In [ ]:
fourth 2

## Quad and Fourth

Abstract the act of applying twice.

In [ ]:
let twice f x = f (f x)

In [ ]:
let quad x = twice double x

In [ ]:
let quad = twice double

In [ ]:
quad 10

## Quad and Fourth

Abstract the act of applying twice.

In [ ]:
let fourth = twice square

In [ ]:
fourth 2

## Applying a function for an arbitrary number of times

Instead of twice, what if I wanted to apply `n` time over an argument where `n` is supplied as an argument

In [ ]:
let rec apply n f x =
  if n = 0 then x
  else f (apply (n-1) f x)

In [ ]:
let quad = apply 0 double

In [ ]:
quad 10

## Higher Order Programming over Lists

<center>

<h1> Map </h1>
<h1> & </h1>
<h1> Fold </h1>
<h4> (sibling of reduce) </h4>
</center>

<center>

<img src="images/map_reduce_new_yorker.png">
</center>

## MapReduce

<h3> "[Google’s MapReduce] abstraction is inspired by the map and reduce primitives present in Lisp and many other <span style="color:orange"> <i> functional languages. </i> </span>" </h3>

<h4 style="text-align:right"> [Dean and Ghemawat, 2008] </h4>


## Map

`List.map` takes a list `[a1; a2; ...; an]` and a higher-order function `f` and returns `[f a1; f a2; ...; f an]`.

## Map


`map (fun x -> shirt_color(x)) [`<img src="images/star_trek.png">`]`

[<span style="color:gold"> Gold </span>; <span style="color:blue"> Blue </span>; <span style="color:red"> Red </span>]

## Map


`map shirt_color [`<img src="images/star_trek.png">`]`

[<span style="color:gold"> Gold </span>; <span style="color:blue"> Blue </span>; <span style="color:red"> Red </span>]

## Map

`List.map` takes a list `[a1; a2; ...; an]` and a higher-order function `f` and returns `[f a1; f a2; ...; f an]`.

In [ ]:
List.map

In [ ]:
List.map (fun x -> x + 1) [1;2;3]

## Map

In [ ]:
let rec map f l =
  match l with
  | [] -> []
  | x::xs -> f x :: (map f xs)

Is there a problem with this implementation?

* Not tail recursive. 
  + Generally not an issue in practice.
  + Recursion depth bound by the size of the list. 

## rev_map

In [ ]:
let rec rev_map f l acc = 
  match l with
  | [] -> acc
  | x::xs -> rev_map f xs (f x::acc)

In [ ]:
let map f l =
  List.rev (rev_map f l [])

In [ ]:
map (fun x -> x + 1) [1;2;3]

## Fold

* Fold is a function for combining elements. 
* Fold is very powerful => very generic / difficult to understand.
* Let's take a simple example first.

In [ ]:
let rec sum_of_elements acc l = 
  match l with
  | [] -> acc
  | x::xs -> sum_of_elements (x + acc) xs
  
let sum_of_elements = sum_of_elements 0

In [ ]:
sum_of_elements [1;2;3;4;5]

## Fold

What is going on here?

```ocaml
let rec sum_of_elements acc l = 
  match l with
  | [] -> acc
  | x::xs -> sum_of_elements (x + acc) xs
  
let sum_of_elements = sum_of_elements 0
```

* There is **traversal** over the shape of the list.
* There is an `accumulator` which keeps track of the current sum so far.
* There is a function **`+`** that is applied to each element and accumulator.
* There is the `initial value` of the accumulator which is `0`.

## Fold

In [ ]:
List.fold_left

* **First argument:** `('a -> 'b -> 'a)` is the function appplied to each element. 
  + `'a` is accumulator and `'b` is current list element
* **Second argument:** `'a` is the initial value of the accumulator.
* **Third argumment:** `'b list` is the list.
* **Result:** `'a` is the value of the accumulator at the end of the traversal.

## Sum of elements using fold_left


```ocaml
let rec sum_of_elements acc l = 
  match l with
  | [] -> acc
  | x::xs -> sum_of_elements (x + acc) xs
  
let sum_of_elements = sum_of_elements 0
```

In [ ]:
List.fold_left (fun acc x -> acc + x) 0 [1;2;3;4;5]

In [ ]:
let rec fold_left f acc l =
  match l with 
  | [] -> acc
  | x::xs -> fold_left f (f acc x) xs

## Fold (left)

as natural transformation of the data structure.

<center>
    
<img src="images/list_shape.svg" width=40% style="float:left">
<img src="images/sum_fold.svg" width=40% style="float:right">
</center>

## fold_right

Fold from the right.

<center>
    
<img src="images/list_shape.svg" width=40% style="float:left">
<img src="images/fold_right.svg" width=40% style="float:right">
</center>

## fold_right

In [ ]:
List.fold_right

In [ ]:
let rec fold_right f l acc = 
  match l with
  | [] -> acc
  | x::xs -> f x (fold_right f xs acc)

* Not tail recursive!

## Behold the power of fold

Any time you need to traverse the list, you can use `fold`.

In [ ]:
let rev l = fold_left (fun acc x -> x :: acc) [] l

In [ ]:
let length l = fold_left (fun acc _ -> acc + 1) 0 l

In [ ]:
let map f l = fold_right (fun x acc -> (f x) :: acc) l []

* `map` is not tail recursive since `fold_right` is not a tail recursive function. 

## Exercise

Implement `exists : ('a -> bool) -> 'a list -> bool` function using `fold_left`. `exists p l` returns `true` if there exists an element `e` in `l` such that `p e` is true. Otherwise, `exists p l` returns `false`.

In [ ]:
let exists p l = failwith "not implemented"

In [ ]:
assert (exists (fun e -> e = 0) [1;3;0] = true)

## Exercise

Implement `append : 'a list -> 'a list -> 'a list` using `fold_right`. 

In [ ]:
let append l1 l2 = failwith "not implemented"

In [ ]:
assert (append [1;2] [3;4] = [1;2;3;4])

<center>

<h1 style="text-align:center"> Fin. </h1>
</center>